In [1]:
import torch

with open('./inputs/day06.txt', 'r') as f:
    m = torch.tensor(
        [list(map(int, line.strip().replace('.', '0').replace('#', '1').replace('^', '2'))) for line in f.readlines()],
        dtype=int
    )

start_x, start_y = map(int, (m==2).nonzero()[0])
m[start_x, start_y] = 0

class LoopError(Exception):
    pass

def get_path(m, start_x, start_y, start_direction='up'):
    positions = []
    x, y, direction = start_x, start_y, start_direction
    while True:
        if (x, y, direction) in positions:
            raise LoopError
        else:
            positions.append((x, y, direction))

        if direction == 'left':
            if y == 0:
                break
            elif m[x, y - 1] == 0:
                y -= 1
            elif m[x, y - 1] == 1:
                direction = 'up'
        elif direction == 'down':
            if x == m.shape[0] - 1:
                break
            elif m[x + 1, y] == 0:
                x += 1
            elif m[x + 1, y] == 1:
                direction = 'left'
        elif direction == 'right':
            if y == m.shape[1] - 1:
                break
            elif m[x, y + 1] == 0:
                y += 1
            elif m[x, y + 1] == 1:
                direction = 'down'
        elif direction == 'up':
            if x == 0:
                break
            elif m[x - 1, y] == 0:
                x -= 1
            elif m[x - 1, y] == 1:
                direction = 'right'

    return positions

part1_path = get_path(m, start_x=start_x, start_y=start_y, start_direction='up')
print('Answer to Day 6, Part 1:', len(set((x, y) for x, y, _ in part1_path)))

part2_ans = 0
for x, y in set((x, y) for x, y, _ in part1_path):
    if (x, y) == (start_x, start_y):
        continue

    m_altered = m.clone()
    m_altered[x, y] = 1

    try:
        get_path(m_altered, start_x=start_x, start_y=start_y, start_direction='up')
    except LoopError:
        part2_ans += 1

print('Answer to Day 6, Part 2:', part2_ans)

Answer to Day 6, Part 1: 4903
Answer to Day 6, Part 2: 1911


In [2]:
with open('./inputs/day07.txt', 'r') as f:
    lines = f.readlines()

def is_good(final_val, numbers):
    current_results = [0]
    for number in numbers:
        next_results = []
        for current_result in current_results:
            if current_result <= final_val:
                next_results += [current_result + number, current_result * number]

        current_results = next_results

    return (final_val in current_results)

part1_ans = 0
for l in lines:
    final_val = int(l.split(':')[0])
    numbers = list(map(int, l.strip().split(':')[1].split()))
    if is_good(final_val, numbers):
        part1_ans += final_val

print('Answer to Day 7, Part 1:', part1_ans)

def is_good_part2(final_val, numbers):
    current_results = [0]
    for number in numbers:
        next_results = []
        for current_result in current_results:
            if current_result <= final_val:
                next_results += [current_result + number, current_result * number, int(str(current_result) + str(number))]

        current_results = next_results

    return (final_val in current_results)

part2_ans = 0
for l in lines:
    final_val = int(l.split(':')[0])
    numbers = list(map(int, l.strip().split(':')[1].split()))
    if is_good_part2(final_val, numbers):
        part2_ans += final_val

print('Answer to Day 7, Part 2:', part2_ans)

Answer to Day 7, Part 1: 1708857123053
Answer to Day 7, Part 2: 189207836795655


In [3]:
import torch
import itertools
import math

with open('./inputs/day08.txt', 'r') as f:
    m = f.read()

lookup_dict = {antennae: i+1 for i, antennae in enumerate(set(m) - set(('\n', '.')))}
m = torch.tensor([list(map(lambda x: lookup_dict.get(x, 0), line)) for line in m.split()], dtype=int)

def is_inbounds(x, y):
    return x >= 0 and y >= 0 and x < m.shape[0] and y < m.shape[0]

def get_antinodes_part1(val):
    antinodes = []
    for a1, a2 in itertools.combinations((m==val).nonzero(), r=2):
        x1, y1 = a1
        x2, y2 = a2
        antinodes.append((int(x1 - (x2 - x1)), int(y1 - (y2 - y1))))
        antinodes.append((int(x2 + (x2 - x1)), int(y2 + (y2 - y1))))

    return set(a for a in antinodes if is_inbounds(a[0], a[1]))

print('Answer to Day 8, Part 1:', len(set(sum((list(get_antinodes_part1(val)) for val in lookup_dict.values()), start=[]))))

def get_antinodes_part2(val):
    antinodes = []
    for a1, a2 in itertools.combinations((m==val).nonzero(), r=2):
        x1, y1 = a1
        x2, y2 = a2
        x_diff = int(x2 - x1)
        y_diff = int(y2 - y1)
        gcd = math.gcd(x_diff, y_diff)
        x_diff = x_diff / gcd
        y_diff = y_diff / gcd

        for i in range(-50, 50):
            antinodes.append((int(x1 + i*x_diff), int(y1 + i*y_diff)))

    return set(a for a in antinodes if is_inbounds(a[0], a[1]))

print('Answer to Day 8, Part 2:', len(set(sum((list(get_antinodes_part2(val)) for val in lookup_dict.values()), start=[]))))

Answer to Day 8, Part 1: 400
Answer to Day 8, Part 2: 1280


In [4]:
with open('./inputs/day09.txt', 'r') as f:
    m = f.read()

file_sizes = [int(m[2*i]) for i in range((len(m) // 2) + 1)]
space_sizes = [int(m[2*i + 1]) for i in range(len(m) // 2)]

def move_yielder():
    back_file_id = len(file_sizes) - 1
    remaining_blocks = file_sizes[back_file_id]
    back_idx = sum(file_sizes) + sum(space_sizes) - 1
    while back_file_id >= 0:
        yield back_file_id
        remaining_blocks -= 1
        back_idx -= 1
        if remaining_blocks == 0:
            back_file_id -= 1
            back_idx -= space_sizes[back_file_id]
            remaining_blocks = file_sizes[back_file_id]

global_yielder = iter(move_yielder())

part1_ans = 0
idx = 0
front_file_id = 0
done = False
while not done:
    for _ in range(file_sizes[front_file_id]):
        part1_ans += front_file_id * idx
        idx += 1
        if idx >= sum(file_sizes):
            done = True
            break

    if done:
        break
    else:
        front_file_id += 1

    for _ in range(space_sizes[front_file_id - 1]):
        back_file_id = next(global_yielder)
        part1_ans += back_file_id * idx
        idx += 1
        if idx >= sum(file_sizes):
            done = True
            break

print('Answer to Day 9, Part 1:', part1_ans)

part2_ans = 0
available_spaces = []
for i in range(len(space_sizes)):
    start_idx = sum(space_sizes[:i]) + sum(file_sizes[:i+1])
    available_spaces.append((start_idx, space_sizes[i]))

for i in range(len(file_sizes)-1, -1, -1):
    orig_start_idx = sum(file_sizes[:i]) + sum(space_sizes[:i])
    
    for j in range(len(available_spaces)):
        idx, available_space = available_spaces[j]
        if file_sizes[i] <= available_space and idx <= orig_start_idx:
            part2_ans += sum((i * (idx + k) for k in range(file_sizes[i])))
            available_spaces[j] = (idx + file_sizes[i], available_space - file_sizes[i])
            break
    else:
        part2_ans += sum((i * (orig_start_idx + k) for k in range(file_sizes[i])))

print('Answer to Day 9, Part 2:', part2_ans)

Answer to Day 9, Part 1: 6307275788409
Answer to Day 9, Part 2: 6327174563252


In [5]:
import torch
import networkx as nx
import itertools

with open('./inputs/day10.txt', 'r') as f:
    m = torch.tensor([[int(c) for c in line.strip()] for line in f.readlines()], dtype=int)

g = nx.Graph()
for i, j in itertools.product(range(m.shape[0]), range(m.shape[1])):
    g.add_node((i, j), height=int(m[i, j]))

    if i > 0:
        g.add_edge((i, j), (i-1, j))
    if j > 0:
        g.add_edge((i, j), (i, j-1))

orig_graph = g.copy()

def get_reachable_nodes(start, visited=[]):
    visited.append(start)
    if g.nodes[start]['height'] == 9:
        return visited

    reachable_nodes = []
    for edge in g.edges(start):
        if g.nodes[edge[1]]['height'] - g.nodes[start]['height'] == 1:
            reachable_nodes += get_reachable_nodes(edge[1], visited)

    return list(set(reachable_nodes))

trailheads = [n for n in g.nodes if g.nodes[n]['height'] == 0]

part1_ans = 0
for trailhead in trailheads:
    part1_ans += sum((int(g.nodes[n]['height'] == 9) for n in set(get_reachable_nodes(trailhead, visited=[]))))

print('Answer to Day 10, Part 1:', part1_ans)

def get_paths(start):
    if g.nodes[start]['height'] == 9:
        return [[start]]

    paths = []
    for edge in g.edges(start):
        if g.nodes[edge[1]]['height'] - g.nodes[start]['height'] == 1:
            for p in get_paths(edge[1]):
                paths.append([start] + p)
    
    return paths

print('Answer to Day 10, Part 2:', sum((len(get_paths(t)) for t in trailheads)))

Answer to Day 10, Part 1: 574
Answer to Day 10, Part 2: 1238
